In [1]:
from agentex import Agentex

client = Agentex(base_url="http://localhost:5003")

In [2]:
AGENT_NAME = "ab080-batch-events"

In [3]:
# (REQUIRED) Create a new task. For Agentic agents, you must create a task for messages to be associated with.
import uuid

rpc_response = client.agents.create_task(
    agent_name=AGENT_NAME,
    params={
        "name": f"{str(uuid.uuid4())[:8]}-task",
        "params": {}
    }
)

task = rpc_response.result
print(task)

Task(id='a2e50f7f-415f-4243-8809-4e4cd22e6f91', created_at=datetime.datetime(2025, 7, 28, 4, 0, 28, 678078, tzinfo=TzInfo(UTC)), name='257301f4-task', status='RUNNING', status_reason='Task created, forwarding to ACP server', updated_at=datetime.datetime(2025, 7, 28, 4, 0, 28, 678078, tzinfo=TzInfo(UTC)))


In [4]:
# Send an event to the agent
from agentex.types import Event
from agentex.types.agent_rpc_params import ParamsSendEventRequest

# The response is expected to be a list of TaskMessage objects, which is a union of the following types:
# - TextContent: A message with just text content   
# - DataContent: A message with JSON-serializable data content
# - ToolRequestContent: A message with a tool request, which contains a JSON-serializable request to call a tool
# - ToolResponseContent: A message with a tool response, which contains response object from a tool call in its content

# When processing the message/send response, if you are expecting more than TextContent, such as DataContent, ToolRequestContent, or ToolResponseContent, you can process them as well

concurrent_event_messages: list[ParamsSendEventRequest] = [
    {
        "content": {"type": "text", "author": "user", "content": "Hello, what can you do?"},
        "task_id": task.id,
    },
    {
        "content": {"type": "text", "author": "user", "content": "Can you tell me a joke?"},
        "task_id": task.id,
    },
    {
        "content": {"type": "text", "author": "user", "content": "What is the capital of France?"},
        "task_id": task.id,
    },
    {
        "content": {"type": "text", "author": "user", "content": "Write a short story about a cat"},
        "task_id": task.id,
    },
    {
        "content": {"type": "text", "author": "user", "content": "Tell me how an LLM works"},
        "task_id": task.id,
    },
]

events: list[Event] = []

for event_message in concurrent_event_messages:
    rpc_response = client.agents.send_event(
        agent_name=AGENT_NAME,
        params={
            "content": {"type": "text", "author": "user", "content": "Hello tell me the latest news about AI and AI startups"},
            "task_id": task.id,
        }
    )

    event = rpc_response.result
    events.append(event)

for event in events:
    print(event)

Event(id='56a214e2-f11a-456d-a65c-ef4128f068b1', agent_id='406f9f42-9f3f-4bb2-869c-d6b36028e487', sequence_id=246, task_id='a2e50f7f-415f-4243-8809-4e4cd22e6f91', content=TextContent(author='user', content='Hello tell me the latest news about AI and AI startups', attachments=None, format='plain', style='static', type='text'), created_at=datetime.datetime(2025, 7, 28, 4, 0, 28, 834958, tzinfo=TzInfo(UTC)))
Event(id='94b21c62-4ad9-46d7-904d-4a2cf034b55e', agent_id='406f9f42-9f3f-4bb2-869c-d6b36028e487', sequence_id=247, task_id='a2e50f7f-415f-4243-8809-4e4cd22e6f91', content=TextContent(author='user', content='Hello tell me the latest news about AI and AI startups', attachments=None, format='plain', style='static', type='text'), created_at=datetime.datetime(2025, 7, 28, 4, 0, 28, 886831, tzinfo=TzInfo(UTC)))
Event(id='5143e12a-7a66-44ad-94cb-bdcfae32770c', agent_id='406f9f42-9f3f-4bb2-869c-d6b36028e487', sequence_id=248, task_id='a2e50f7f-415f-4243-8809-4e4cd22e6f91', content=TextContent

In [5]:
from agentex.lib.utils.dev_tools import subscribe_to_async_task_messages

task_messages = subscribe_to_async_task_messages(
    client=client,
    task=task, 
    only_after_timestamp=event.created_at, 
    print_messages=True,
    rich_print=True,
    timeout=20,
)

╭──────────────────────── AGENT [07/28/2025 04:00:38] ─────────────────────────╮
│ Processed event IDs: ['56a214e2-f11a-456d-a65c-ef4128f068b1',                │
│ '94b21c62-4ad9-46d7-904d-4a2cf034b55e']                                      │
╰──────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────── AGENT [07/28/2025 04:00:54] ─────────────────────────╮
│ Processed event IDs: ['5143e12a-7a66-44ad-94cb-bdcfae32770c',                │
│ 'ac2cdee3-00eb-4294-8be8-cb30c35664e3',                                      │
│ 'fba14bc1-af91-41e5-bc83-f892d0e2d8a2']                                      │
╰──────────────────────────────────────────────────────────────────────────────╯

Streaming timed out after 20 seconds - returning collected messages
